In [ ]:
import gzip
import csv
import random
from copy import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV
from greenpyce.feature_engineering import TargetEncoder
from greenpyce.feature_engineering import onehot
from greenpyce.feature_engineering import RankCategorical
from greenpyce.feature_engineering import LabelCount
from greenpyce.feature_engineering import time_count_between_dates
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# The universe and everything else
random.seed(42)

## Helper functions

In [ ]:
def complete_report(y_test, y_pred):
    print("*********************************")
    print("Accuracy: {0:.2f}".format(accuracy_score(y_test, y_pred)))
    print("Precision: {0}".format(metrics.precision_score(y_test, y_pred)))
    print("Recall: {0}".format(metrics.recall_score(y_test, y_pred)))
    print("F1-Score: {0}".format(metrics.f1_score(y_test, y_pred)))
    conf_m = confusion_matrix(y_test, y_pred)
    print(conf_m)
    try: 
        print("Roc Area: {0}".format(metrics.roc_auc_score(y_test, y_pred))) 
        roc_curve_label(y_test, y_pred)
    except:
        print("--")
    print("*********************************")
    
    return metrics.roc_auc_score(y_test, y_pred)
        
def roc_curve_label(y_test, y_pred):
    plt.figsize=(10,6)
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='RT + LR')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    
    
def expand_date_info(df, columns, hour_feats=False):
    """
    Create year, month, day, hour and minute from a datetime column.
    """
    for column in columns:
        df[column + "_DAY"] = df[column].apply(lambda x: x.day)
        df[column + "_MONTH"] = df[column].apply(lambda x: x.month)
        df[column + "_YEAR"] = df[column].apply(lambda x: x.year)
        if hour_feats:
            df[column + "_HOUR"] = df[column].apply(lambda x: x.hour)

    return df

def parse_date(x):
    text = ""
    year = int(x / 10000)
    month = int((x % 10000) / 100)
    day = (x % 100)
    text = str(year) + "-" + str(month) + "-" + str(day)
    
    return text


def build_kaggle_submission(model, df_test, features, filename="my_submission"):
    """
    Build a compressed Kaggle Submission
    
    df_test : Dataframe com os dados de teste
    model : Modelo treinado
    features : Lista de features usadas para o treinamento
    filename : Nome do arquivo de submissão
    """
    
    preds = model.predict_proba(df_test[features])
    preds = preds[:, 1]
    
    with gzip.open(filename + '.csv.gz', 'wt') as outf:
        fo = csv.writer(outf, lineterminator='\n')
        fo.writerow(["sample_id", "is_listened"])       
        for i, pred in enumerate(preds):
            fo.writerow([i, pred])

## GridSearch definition

In [ ]:
def optimize_model(model, X, y, params_grid, cross_validation):
    model_tmp = copy(model)
    search = GridSearchCV(estimator=model_tmp, param_grid=params_grid, cv=cross_validation)
    search.fit(X, y)
    
    print("Best score: ", search.best_score_)
    print("Best params: ", search.best_params_)
    print("Function used to choose the best parameter for the model: ", search.scorer_)
    
    # input the best params inside the model and return the model with the right params
    best_params = search.best_params_
    model_tmp = model_tmp.set_params(**best_params)
    return model_tmp, best_params

## Feature engineering

### User specific features

In [ ]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
nmidia_feats = pd.read_csv("../features/nmidia_feats.csv", sep=";")

In [ ]:
nmidia_feats["user_id"] = nmidia_feats["index"]

In [ ]:
user_features_used = ["user_id", "nartist_with_flow", "nartist_with_flow_listened", "nmidia_regular_listened", "nmidia_regular",
                      "nmidia_with_flow", "nartist_regular_listened", "nartist_regular", "nmidia_with_flow_listened"]
df_user_feats = nmidia_feats[user_features_used]

In [ ]:
df_train = pd.merge(df_train, df_user_feats, on="user_id")
df_test = pd.merge(df_test, df_user_feats, on="user_id")
df_test.head()

In [ ]:
df_train["year"] = df_train["release_date"].apply(lambda x: x // 10000)
df_train.drop(df_train[df_train["year"] > 2017].index , inplace=True)
df_train["text"] = df_train["release_date"].apply(lambda x : parse_date(x))
df_train["text"] = df_train["text"].apply(lambda x : pd.to_datetime(x))
df_train["release_date"] = df_train["text"]
expand_date_info(df_train, ["release_date"]).head()


df_test["year"] = df_test["release_date"].apply(lambda x : x // 10000)
df_test.drop(df_test[df_test["year"] > 2017].index , inplace=True)
df_test["text"] = df_test["release_date"].apply(lambda x : parse_date(x))
df_test["text"] = df_test["text"].apply(lambda x : pd.to_datetime(x))
df_test["release_date"] = df_test["text"]
expand_date_info(df_test, ["release_date"]).head()

In [ ]:
df_train.drop(["text", "release_date"], axis=1, inplace=True)
df_test.drop(["text", "release_date"], axis=1, inplace=True)

In [ ]:
df_train["media_duration"] = df_train["media_duration"].apply(lambda x: round(x / 60.0))
df_test["media_duration"] = df_test["media_duration"].apply(lambda x : round(x / 60.0))

In [ ]:
cols_for_target_encoder = ["genre_id", "context_type", "user_id", "media_id", "media_duration"]
cols_for_onehot = ["platform_name", "platform_family", "listen_type", "user_gender", "user_age"]
cols_for_rank = ["artist_id"]

target = "is_listened"

In [ ]:
te = TargetEncoder(cols_for_target_encoder, "is_listened", inplace=True)
te.fit(df_train)

te.transform(df_train)
te.transform(df_test)

In [ ]:
rc = RankCategorical(cols_for_rank)
rc.fit(df_train)
rc.transform(df_train)
rc.transform(df_test)

In [ ]:
df_train = onehot(df_train, cols_for_onehot)
df_test = onehot(df_test, cols_for_onehot)

In [ ]:
features = [col for col in df_train.columns.values if col != target]
features

## RandomForestClassifier params to be used in GridSearch

parametros utilizados no modelo:

    - n_estimators = numero de arvores
    - criterion = funcao que mede a qualidade dos splits. Opções são gini que mede a impureza Gini e "entropy" que mede o ganho de informação.
    - max_features = o numero de features que deve ser considerada para cada split
    - max_depth = profundidade das arvores
    - oob_score = utiliza leave-one-out para calcular acurácia de validação fora da amostra   

In [ ]:
grid_params = {"n_estimators": [3, 6, 9, 12, 15], 
               "criterion": ["gini", "entropy"],
               "max_features": ["auto", "sqrt", "log2", None],
               "max_depth": [5, 10, 15, 20, 30, None],
               "oob_score": [False, True],
               }

In [ ]:
X = df_train[features]
y = df_train[target]

model = RandomForestClassifier()
best_model, best_params = optimize_model(model, X, y, grid_params, 10)

## Aplicação do modelo RandomForest com os melhores parametros encontrados pelo GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train[features], df_train[target], test_size = 0.2)
best_model.fit(X_train, y_train)
best_model.score(X_test, y_test)
complete_report(y_test, best_model.predict(X_test))